## Set Up Environment

In [1]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [4]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
with open("keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("keystores/staking_test.json", key_data)

In [98]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
783954813.6165906


In [97]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
1052231.256087


### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server
# for the first time since reinitializing.

# send_icx(wallets[20].get_address(), 2000 * ICX, btest_wallet)

# transaction = TransactionBuilder()\
#     .from_(wallet.get_address())\
#     .to(btest_wallet.get_address())\
#     .value(1000000 * ICX)\
#     .step_limit(1000000) \
#     .nid(NID) \
#     .nonce(2) \
#     .version(3) \
#     .build()
# signed_transaction = SignedTransaction(transaction, wallet)
# tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

### Deployment

In [ ]:
old_dividends = 'cx13f08df7106ae462c8358066e6d47bb68d995b6d'

In [ ]:
# Cell 5
# The following addresses are those deployed to the mainnet.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx66d4d90f5f113eba575bf793570135f9b10cece1'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx43e2eec79eb76293c298f2b17aec06097be606e0'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx203d9cd2a669be67177e997b8948ce2c35caffae'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxf58b9a1898998a31be7f1d99276204a3333ac9b3'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx835b300dcfe01f0bdb794e134a0c5628384f4367'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx10d59e8103ab44635190bd4139dbfd682fa2d07e'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxa0af3165c08318e988cb30993b3048335b94af6c'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx40d59439571299bca40362db2a7d8cae5b0b30b0'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx44250a12074799e26fdeee75648ae47e2cc84219'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxe647e0af68a4661566f5e9861ad4ac854de808a2'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx2609b924e33ef00b648a409245c7ea394c467824'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx88fd7df7ddff82f7cc735c871dc519838cb235bb'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxf61cd5a45dc9f91c15aa65831a30a90d59a09619'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxcfe9d1f83fa871e903008471cca786662437e58d'}}

In [ ]:
# Cell 5a
## Testnet Yeouido

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3259f3ff9a51ca3bf170d4ff9104cf4af126ca1c'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cx4c456f4a02d2576fe712ea10b311a5fe8d06d205'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx5b996d251009340f7c312b9df5c44f0f39a20a91'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx1754666c6779dc5e495a462144dd15e4a68fe776'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx430955c5a5e2a6e48c1f2aaa7258dc4c84222247'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx2e3398dfce78a3c83de8a41d7c5f4aa40d3a4f30'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx483630769b61b76387d6ed90c911c16da546784f'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxc48c9c81ceef04445c961c5cc8ff056d733dfe3a'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx4a1179d3ff8c692c4b20bfab2a2038ef79811002'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cxa40ff2a028d4def5ae12ae82a188ad4eb051ab1f'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx36169736b39f59bf19e8950f6c8fa4bfa18b710a'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx5d886977b7d24b9f73a460c9ca2d43847997c285'}}

In [24]:
# Cell 5c
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx8b77a58aaf15499c5b774a84f514edf101e78911'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx53cdc86fb50830d77143c077c91a3133b7ab46e7'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cx5d886977b7d24b9f73a460c9ca2d43847997c285'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxfcfe26c43a7e4a9e875826bc943901c2f458fda7'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx0263f8cb222e965c2810b917c0ffc54e4aed9a5e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx44e80f46842a009b3371b07a8a0782f0e11fd0a8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx77dc1982c949e7ce5bf92260974ef8b838357964'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx2bec556f4d890677d54f9a4b2ab8e648496376ae'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxe90e668a8467b951d41220575674a6cd17fafddb'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx236877888d2dd48ab327612146254796e516e71a'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx52887aff55673d63c8bbae0499a8b93024ec2af1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxa28ed6e7ec8f5cc7a9bce10f72a0725347675fcb'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx7896b19a958b91cbb6cef37dea942cdaaaba7756'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx0250899375082465b9b965b69585094feecfcc94'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx049b2b12783cc5796041c9a725d3e12a372035d9'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx8fa567dfd5e41c79c5c56e0d876d43579d57c885'},
 'rebalancing': {'zip': 'token_contracts/rebalancing.zip',
  'SCORE': 'cx8fa567dfd5e41c79c5c56e0d876d43579d57c885'}}

In [19]:
print(json.dumps({contract: contracts[contract]['SCORE'] for contract in contracts}))

{"loans": "cx6b8a849a32334268a175d0865f1b6c7266aee24e", "staking": "cxeb16f219b387d246c43b7f44dfcc980012d0f39b", "router": "cxf0b20614f3cf0731e9f154510194ade52049334c", "dividends": "cxdcf29468e57894a02efa6228e2074a4dab6e2a00", "reserve": "cxe2c4cff8fc36cd131e7169f5837ff46c4b646a54", "daofund": "cx2a11c4ecdb2d307a3b09e1d8a7d8afbfeff9b21a", "rewards": "cxf34b126e6e047d881e00d15a29af894a48a1f2a6", "dex": "cx9485e2be64b74183cc86b54df5d3fb7a14ab6f86", "governance": "cx20737d71f8c1c5b5bb343389e1188696f9c83df0", "oracle": "", "sicx": "cx1b8d344acef77395da5f372bbbf8d3d3de76efc6", "bnUSD": "cx4c202f0d4da93cc78f444e80804753b0e310000d", "bnXLM": "cxd25cb82b63d3bbb02b6c06f9f58de27b89734efe", "bnDOGE": "cxee2c96bccc9b3b894322c5f2e6629067d85457f2", "baln": "cx044e27e4fbb71cf8e94468c4624390cbb54b04cb", "bwt": "cx4028a1dd80a34cf9cdd8bfe8690939900a7d29e8"}


In [ ]:
for score in contracts:
    contracts[score]['SCORE'] = {"loans": "cx784f198d1b78fa12912fbda6edc38d9fb627067a", "staking": "cxc3733d7112d27e51490067af4f486da9380f1a41", "dividends": "cxd47dc3297a03f8245310994ccb5f7c166f1562e5", "reserve": "cx0af7adaef9fde9ce6dca17632dd5bbe6b0cb88c8", "daofund": "cxace1660691d27d12f638352fa6977329d3be9ab9", "rewards": "cxa81b648378628115071d216fbd7ba2e3657377b1", "dex": "cxc428e09c5f8783b0676e61ce83ca16bf6d7682fd", "governance": "cxfbec559c11a167191cdd88a9c1d4a292514a00cf", "oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964", "sicx": "cx196fda5daa2de632d190b869a8501693ed93d3b7", "bnUSD": "cx38aa1937a1da7031818ba68e2b9feb5fff3235ef", "bnXLM": "cx9c6514df52341309b15da134168d87e81782bd72", "bnDOGE": "cx8907cdcdaadcdbee318fcefd89ccab953b4e7a38", "baln": "cxe18cd2527e406cc6d79f4f40ff258ea0eb236219", "bwt": "cxbea02ccf3110273b4538af61e697eb6ff4eea156"}[score]
contracts

## Deploy All SCOREs

In [104]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res

def fast_send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

def deploy_all(wallet, staking_wallet, deploy_dummy_oracle = False):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]

    if deploy_dummy_oracle and network == "mainnet":
        raise Exception("Cannot deploy dummy oracle on mainnet")

    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    
    if deploy_dummy_oracle:
        res = deploy_SCORE(contracts['oracle'], {}, wallet, 0)
        results[f'{contracts["oracle"]}|deploy|{{}}'] = res
        oracle = res.get('scoreAddress', '')
        contracts['oracle']['SCORE'] = oracle
    
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    print(config)
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate',
             'params': {'_delegations': [{'_address': prep, '_votes_in_per': 100 * ICX // len(preps)} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results


def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result


In [105]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet, True)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

core_contracts/governance.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/oracle.zip
Retrying in 1 seconds...
Status: 1

core_contracts/loans.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/router.zip
Retrying in 1 seconds...
Status: 1

core_contracts/dividends.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/reserve.zip
Retrying in 1 seconds...
Status: 1

core_contracts/daofund.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/rewards.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/dex.zip
Retrying in 1 seconds...
Status: 1

token_contracts/bnUSD.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

token_contracts/bnXLM.zip
Retrying in 1 seconds...
Status: 1

token_contracts/bnDOGE.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

token_contracts/baln.zip
Retrying in 1 seconds...
Status: 1

token_contrac

In [86]:
# Cell 7d
# Reset all addresses through governance.

addresses = {"loans": "cx66d4d90f5f113eba575bf793570135f9b10cece1",
             "staking": "cx43e2eec79eb76293c298f2b17aec06097be606e0",
             "dividends": "cx203d9cd2a669be67177e997b8948ce2c35caffae",
             "reserve": "cxf58b9a1898998a31be7f1d99276204a3333ac9b3",
             "daofund": "cx835b300dcfe01f0bdb794e134a0c5628384f4367",
             "rewards": "cx10d59e8103ab44635190bd4139dbfd682fa2d07e",
             "dex": "cxa0af3165c08318e988cb30993b3048335b94af6c",
             "oracle": "cxe647e0af68a4661566f5e9861ad4ac854de808a2",
             "sicx": "cx2609b924e33ef00b648a409245c7ea394c467824",
             "bnUSD": "cx88fd7df7ddff82f7cc735c871dc519838cb235bb",
             "baln": "cxf61cd5a45dc9f91c15aa65831a30a90d59a09619",
             "bwt": "cxcfe9d1f83fa871e903008471cca786662437e58d"}

send_tx('governance', 0, 'setAddresses', {'_addresses': addresses}, btest_wallet)
send_tx('governance', 0, 'setAdmins', {}, btest_wallet)
send_tx('governance', 0, 'setContractAddresses', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx66d4d90f5f113eba575bf793570135f9b10cece1', 'staking': 'cx43e2eec79eb76293c298f2b17aec06097be606e0', 'dividends': 'cx203d9cd2a669be67177e997b8948ce2c35caffae', 'reserve': 'cxf58b9a1898998a31be7f1d99276204a3333ac9b3', 'daofund': 'cx835b300dcfe01f0bdb794e134a0c5628384f4367', 'rewards': 'cx10d59e8103ab44635190bd4139dbfd682fa2d07e', 'dex': 'cxa0af3165c08318e988cb30993b3048335b94af6c', 'oracle': 'cxe647e0af68a4661566f5e9861ad4ac854de808a2', 'sicx': 'cx2609b924e33ef00b648a409245c7ea394c467824', 'bnUSD': 'cx88fd7df7ddff82f7cc735c871dc519838cb235bb', 'baln': 'cxf61cd5a45dc9f91c15aa65831a30a90d59a09619', 'bwt': 'cxcfe9d1f83fa871e903008471cca786662437e58d'}} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 second

KeyboardInterrupt: 

In [106]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

['loans', 'staking', 'router', 'dividends', 'reserve', 'daofund', 'rewards', 'dex', 'governance', 'oracle', 'sicx', 'bnUSD', 'bnXLM', 'bnDOGE', 'baln', 'bwt', 'rebalancing']
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx0b13453906786489c7989d464f33ed6dc33256bc', 'staking': 'cx2d9ec07136fe5d1a7173e55b0773b41d878db911', 'router': 'cx738ad566f1bce722a0919158a64645e3e5627cb7', 'dividends': 'cx34a2127f054d26b3b3b128e6243776c67b0ee9ba', 'reserve': 'cx84a485d66094f0edb59f2cb8177c2319b212bb54', 'daofund': 'cx71f07ad9097b5ad99949a093eaeacd25682f0b66', 'rewards': 'cxc234187b47c16f9f3077a5f4dcc8856ee9124dcf', 'dex': 'cx337517d2459fd0abf52e4662bcfb0d872214171b', 'oracle': 'cxebb9a5e5abfacccd978c19e38e5dae6da7c6f5cb', 'sicx': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'bnUSD': 'cx773b203256491228aa05aff96fd9357e388f59ef', 'baln': 'cx459a405a8a5450033b5ba77fce337b

In [107]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx0b13453906786489c7989d464f33ed6dc33256bc', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 ******************

In [108]:
send_tx('oracle', 0, 'rig_the_market', {'_base': 'USD', '_quote': 'ICX', '_rate': ICX // 2}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling rig_the_market, with parameters {'_base': 'USD', '_quote': 'ICX', '_rate': 500000000000000000} on the oracle contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************


{'txHash': '0x9df9257a6e65891644536e39aed608749d4e8666e3a357f11be3d877303064f7',
 'blockHeight': 4590660,
 'blockHash': '0x080a44aed3dfb83366780c3c27651b1bebc71834863d7de76533d14be4e54c48',
 'txIndex': 1,
 'to': 'cxebb9a5e5abfacccd978c19e38e5dae6da7c6f5cb',
 'stepUsed': 145760,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 145760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [35]:
# Cell 7c
# Set up and fund sICX/bnUSD market

send_tx('governance', 210 * ICX, 'createBnusdMarket', {}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling createBnusdMarket, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx66ebc45553e164f2463575ea5bb817022c55f85c', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cxa644b8897c9cedc49290d30bb22c85fd870c74f0'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0', 'cx5db663c48d12c56033a8c639a041a20c6df89ea2', '0x1a055690d9db80000'], 'data': []} 

{'scoreAddress': 'cx1027c16288194e933a913e806ce1744ba581f9b8', 'indexed': ['Mint(Address,int,bytes)', 'c

{'txHash': '0xf62eb72b1767ba69dc79453b07f5fd7a781779f45ff420249ee7658505fa4e9e',
 'blockHeight': 4443614,
 'blockHash': '0x921f7b636d69ba2422983ff20c69bb0bb796533afbb6e52b7a301a5acdbb6ddb',
 'txIndex': 1,
 'to': 'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0',
 'stepUsed': 2294780,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2294780,
 'eventLogs': [{'scoreAddress': 'cx66ebc45553e164f2463575ea5bb817022c55f85c',
   'indexed': ['OraclePrice(str,str,Address,int)',
    'USDICX',
    'BandChain',
    'cxa644b8897c9cedc49290d30bb22c85fd870c74f0'],
   'data': ['0x84c5d90848544bb']},
  {'scoreAddress': 'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0',
    'cx5db663c48d12c56033a8c639a041a20c6df89ea2',
    '0x1a055690d9db80000'],
   'data': []},
  {'scoreAddress': 'cx1027c16288194e933a913e806ce1744ba581f9b8',
   'indexed': ['Mint(Address,int,bytes)',
    'cx9927d0858af40d9875a1dbef4c0dab4f5857aea0'

### Set up BALN/bnUSD Market
Be sure to drive transactions to update to the next day before calling this.

In [ ]:
# Cell 7d
# Set up and fund BALN/bnUSD market

send_tx('governance', 0, 'createBalnMarket', {'_bnUSD_amount': 20 * ICX, '_baln_amount': ICX}, btest_wallet)

In [ ]:
# Hedge Tests
# 2 borrowers, different debt for each borrower
# Each borrower supplies liquidity in their proportion of the debt
# knock the ratio off by 5 percent, then rebalance

## Deploy or Update a single SCORE

In [18]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'oracle'
update = 1
params = {}
if update == 0 and contract_name != 'governance' and contract_name != 'oracle':
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    res = deploy_SCORE(contract, params, btest_wallet, update)
print(res['scoreAddress'])

core_contracts/oracle.zip


JSONRPCException: {'code': -32602, 'message': "JSON schema validation error: 'to' has an invalid value"}

In [ ]:
# Update all

compress()
for contract in contracts:
    deploy_SCORE(contract, {}, btest_wallet, 1)

In [14]:
print(contracts['oracle']['SCORE'])

## Test of Typical Flow

In [ ]:
# Cell 9
# 1. staking.stakeICX() - Get 800 ICX worth of sICX from Staking
# 2. loans.depositAndBorrow() - Deposit 800 ICX and Mint 50 bnUSD Loan
# 3. loans.depositAndBorrow() - Just deposit 30 ICX collateral
# 4. loans.depositAndBorrow() - Originate Loan of 10 bnUSD
# 5. loans.returnAsset() - Repay 5 bnUSD
# 6. loans.withdrawCollateral() - Withdraw 25 sICX Collateral
# 7. depositAndBorrow() via sICX.transfer() - Add 200 sICX collateral directly
# Print results1 if there are any failures.

txns = [{'contract': 'staking', 'value': 800*ICX, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': 800*ICX, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': 50 * ICX}},
        {'contract': 'loans', 'value': 30*ICX, 'method': 'depositAndBorrow', 'params': {'_asset': '', '_amount': 0}},
        {'contract': 'loans', 'value': 0, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': 10 * ICX}},
        {'contract': 'loans', 'value': 0, 'method': 'returnAsset', 'params': {'_symbol': 'bnUSD', '_value': 5 * ICX}},
        {'contract': 'loans', 'value': 0, 'method': 'withdrawCollateral', 'params': {'_value': 25 * ICX}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['loans']['SCORE'], '_value': 200 * ICX, '_data': json.dumps({"_asset": "", "_amount": 0}).encode()}}]

results1 = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    results1[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

### Add new assets

In [ ]:
# Not totally working yet since the Oracle needs to be set through the governance SCORE.
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnDOGE']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnXLM']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('bnDOGE', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)
# send_tx('bnXLM', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)


### Test Liquidation

In [ ]:
# Cell 10
# 1. Deposit collateral to fresh wallet
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.
# 3a. Mints 200 bnUSD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' bnUSD and have standing of 'Liquidate'.
# Test methods have been removed.

# send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
# send_tx('loans', 170 * ICX, 'depositAndBorrow', {'_asset': '', '_amount': 0}, btest_wallet)
# call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
# send_tx('loans', 0, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 200 * ICX}, btest_wallet)
# call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
# send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)


In [ ]:
# Cell 11
# 4a. If the account does not yet have a standing of 'Liquidate' add another loan of '0xad78ebc5ac6200000'.
# Test methods have been removed.

# send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
# send_tx('loans', 1, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 200 * ICX}, btest_wallet)
# send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
# call_tx('loans', 'checkDeadMarkets', {})
# print(call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()}))


In [ ]:
# Cell 12
# 5. Check the account standing after depositing collateral - Should be 'No Debt'.
# 6. Liquidate the account position.
# 7. Check the standing of the account after liquidation. Should now have zero balance for sICX and bnUSD.
# 8. Checking the debts should show the sum of the borrower debt and the bad debt equals the total supply.

call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'liquidate', {'_owner': btest_wallet.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


## Retire some debt to get bnUSD out of Dead Market state

In [83]:
# Cell 13
# 1. Check that there is a Dead Market
# 2. Check the bad debt, liquidation pool and dead_market state of the asset.
# 3. Retire enough of the bad debt to get the asset out of the dead_market state.
# 4. Again, check the bad debt, liquidation pool and dead_market state of the asset.

call_tx('loans', 'getMaxSellAmount', {})
call_tx('loans', 'getAvailableAssets', {})
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 200 * ICX}, btest_wallet)
call_tx('loans', 'getAvailableAssets', {})


------------------------------------------------------------------------------------------------------------------
Reading getMaxSellAmount, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'sICX': '0x0', 'bnUSD': '0x0'}
------------------------------------------------------------------------------------------------------------------
Reading getAvailableAssets, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'bnUSD': {'symbol': 'bnUSD', 'address': 'cxdc12ea63afd40927bec3fdf5b45cff88b4a729bb', 'peg': 'USD', 'added': '0x5cea0f0c3b2e4', 'is_collateral': '0x0', 'active': '0x1', 'borrowers': '0x2', 'total_supply': '0x106cf83ea31f5c0000', 'total_burned': '0x0', 'bad_debt': '0x0', 'liquidation_pool': '0x0', 'dead_market': '0x0'}}
------------------------------------------------

{'bnUSD': {'symbol': 'bnUSD',
  'address': 'cxdc12ea63afd40927bec3fdf5b45cff88b4a729bb',
  'peg': 'USD',
  'added': '0x5cea0f0c3b2e4',
  'is_collateral': '0x0',
  'active': '0x1',
  'borrowers': '0x2',
  'total_supply': '0x106cf83ea31f5c0000',
  'total_burned': '0x0',
  'bad_debt': '0x0',
  'liquidation_pool': '0x0',
  'dead_market': '0x0'}}

## Define a Vote

In [ ]:
send_tx('governance', 0, 'defineVote', {'name': 'Enable Divs', 'quorum': 20, 'vote_start': 40, 'duration': 5, 'snapshot': 40, 'actions': '"enable_dividends": {"_status": True}'}, btest_wallet)

In [ ]:
call_tx('governance', 'checkVote', {'_vote_index': 1})

In [ ]:
description = "Add bnBTC as an asset."
send_tx('governance', 0, 'defineVote', {'name': 'Add bnBTC', 'description': description, 'quorum': 20, 'vote_start': 84, 'duration': 5, 'snapshot': 83, 'actions': '{"add_asset": {"_symbol": "bnBTC"}}'}, btest_wallet)

In [ ]:
send_tx('governance', 0, 'activateVote', {'name': 'Add bnBTC'}, btest_wallet)

In [ ]:
call_tx('governance', 'checkVote', {'_vote_index': 2})

In [ ]:
call_tx('governance', 'checkVote', {'_vote_index': 3})

In [ ]:
description = 'When you borrow Balanced Dollars, you pay a 1% origination fee. The implied annual interest rate (1%) is still only half that of other platforms. A small increase to 1.15% will create more income for Balanced and BALN holders, while limiting the inconvenience to borrowers.'
send_tx('governance', 0, 'defineVote', {'name': 'BIP3: Increase origination fee to 1.15%', 'description': description, 'quorum': 20, 'vote_start': 110, 'duration': 5, 'snapshot': 109, 'actions': '{"update_origination_fee": {"_fee": 115}}'}, btest_wallet)

### Cancel a vote

In [ ]:
send_tx('governance', 0, 'setAddressesOnContract', {'_contract': 'router'}, btest_wallet)

### Check balanceOfAt method

In [ ]:
call_tx("router", "getDex", {})

### Unstake some sICX

In [ ]:
send_tx('sicx', 0, 'transfer', {'_to': contracts['staking']['SCORE'], '_value': 50 * ICX, '_data': json.dumps({"method": "unstake", "params": {}}).encode()}, btest_wallet)

### Testing BALW distributions and Transfers

In [ ]:
for i in range(5):
    print(wallets[i].get_address())

In [ ]:
for i in range(5):
    send_tx('bwt', 0, 'transfer', {'_to': wallets[i].get_address(), '_value': 20000000}, btest_wallet)

In [ ]:
send_tx('bwt', 0, 'distribute', {}, btest_wallet)

In [ ]:
btest_wallet.get_address()

## Set redemption fee

In [ ]:
send_tx('governance', 0, 'setRedeemBatchSize', {'_value': 10}, btest_wallet)

## Tests with multiple wallets and redemptions

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [ ]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [ ]:
for i in range(20):
    print(f'{wallets[i].get_address()}')

In [ ]:
for i in range(120, 200):
    send_icx(wallets[i].get_address(), 2000 * ICX, btest_wallet)

In [ ]:
for i in range(200):
    print(f'{i:3d}: {wallets[i].get_address()}: {icon_service.get_balance(wallets[i].get_address()) / 10**18: 9.3f}')

In [ ]:
for i in range(20):
    print(int(call_tx("bnUSD", "balanceOf", {"_owner": wallets[i].get_address()}), 0) / 10**18)

In [ ]:
# 10000 ICX into each wallet. Uncomment if needed. These wallets are already more than well stocked enough on the tbears server.

# for i in range(20):
#     transaction = TransactionBuilder()\
#         .from_(wallet.get_address())\
#         .to(wallets[i].get_address())\
#         .value(10 * ICX)\
#         .step_limit(1000000) \
#         .nid(NID) \
#         .nonce(2) \
#         .version(3) \
#         .build()
#     signed_transaction = SignedTransaction(transaction, wallet)
#     tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

In [ ]:
# Mint some bnUSD to first 70 wallets.

for i in range(60, 100):
    res = send_tx('loans', 500 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 100 * ICX}, wallets[i])

In [ ]:
send_tx('bnUSD', 0, 'transfer', {'_to': wallet.get_address(), '_value': 50 * ICX}, wallets[19])
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 65 * ICX}, wallet)

In [ ]:
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 50 * ICX}, wallet)

In [ ]:
# Pay off debt in full.

for i in range(5, 10):
    params = {'_to': wallets[i+10].get_address(), '_value': 50 * ICX}
    res = send_tx('bnUSD', 0, 'transfer', params, wallets[i])
    sleep(2)
    res = send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 110 * ICX}, wallets[i+10])
    sleep(1)

In [ ]:
# Drive rewards distributions

for i in range(200):
    fast_send_tx('rewards', 0, 'distribute', {}, btest_wallet)
    sleep(.4)


In [ ]:
res = send_tx('loans', 500 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 100 * ICX}, wallets[90])

In [ ]:
for i in range(15, 20):
    res = send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 15 * ICX}, wallets[i+100])
    sleep(1)

In [ ]:
# redeem 10 bnUSD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(60, 80):
    params = {'_to': wallets[i+100].get_address(), '_value': 20 * ICX}
    res = send_tx('bnUSD', 0, 'transfer', params, wallets[i])
    sleep(2)
    res = send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 15 * ICX}, wallets[i+100])
    sleep(1)

In [ ]:
for _ in range(10):
    send_tx('loans', 0, 'depositAndBorrow', {'_asset': '', '_amount': 0}, btest_wallet)
    sleep(1)

### Add a new asset

In [ ]:
send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnXLM']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)

In [ ]:
send_tx('governance', 0, 'setAssetOracle', {'_symbol': 'bnXLM', '_address': contracts['oracle']['SCORE']}, btest_wallet)

In [ ]:
# Mint some bnXLM to some wallets.

for i in range(10, 30):
    res = send_tx('loans', 500 * ICX, 'depositAndBorrow', {'_asset': 'bnXLM', '_amount': 300 * ICX}, wallets[i])


In [ ]:
params = {'_holders': [wallet.get_address(), btest_wallet.get_address()]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

In [ ]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

borrowerCount = int(call_tx('loans', 'borrowerCount', {}, False), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    position = call_tx('loans', 'getPositionByIndex', {'_index': i, '_day': -1}, False)
    addresses.append(position['address'])

holders = call_tx('rewards', 'getBalnHoldings', {'_holders': addresses}, False)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    result = int(call_tx('baln', 'balanceOf', {'_owner': contracts[contract]['SCORE']}, False), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 70):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'BALN/bnUSD', '_snapshot_id': 1, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['sicx']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wall in [wallet, btest_wallet]:
    address = wall.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = int(result, 0) // 10**18
print(balances)

## Check getDebts for dividends

In [ ]:
debts = call_tx('loans', 'getDebts', {'_address_list': [call_tx('loans', 'getPositionAddress', {'_index': idx}) for idx in range(5, 10)], '_day': 1})
for address in debts:
    print(f'{address}: {int(debts[address], 0) / 10**18}')

### Check bonusDist method

In [ ]:
send_tx('baln', 0, 'transfer', {'_to': 'cxde0d2b42b55b4e9305affc21dc4e7a4f01acf014', '_value': 10**18}, wallet)

In [ ]:
send_tx('governance', 0, 'bonusDist', {'_addresses': [wallet.get_address(), btest_wallet.get_address()], '_amounts': [10**21, 10**21]}, btest_wallet)

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
contracts['dex']['SCORE']

In [14]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 29

path = [contracts['sicx']['SCORE'], contracts['bnUSD']['SCORE']]

params = {'_path': path}

print(json.dumps(params))
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['router']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("route")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'sICX/ICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

### sICX / bnUSD Pool

In [109]:
# Create a new pool and generate assets for it

sICX_amount = 50 * ICX
bnUSD_amount = 100 * ICX
icx_price = 2
sICX_contribute_amount = sICX_amount
bnUSD_contribute_amount = bnUSD_amount

txns = [{'contract': 'staking', 'value': sICX_amount, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': int(icx_price * 6 * bnUSD_amount), 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], btest_wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res
    
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params({'_owner': btest_wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
print(result)
btest_debt = int(result["total_debt"],0)/ICX
btest_collateral = int(result["collateral"], 0)/ICX

result = call_tx('dex', 'getPoolStats', {'_id': 2})
total_sicx = int(result['quote'], 0) / ICX
total_lp = int(result['total_supply'], 0)

user_lp = int(call_tx('dex', 'balanceOf', {'_owner': btest_wallet.get_address(), '_id': 2}), 0)

btest_sicx_lp = total_sicx * user_lp / total_lp

btest_total_sicx = btest_collateral + btest_sicx_lp

print(f"Debt: {btest_debt}, collateral: {btest_collateral}, lp sicx: {btest_sicx_lp}, total: {btest_total_sicx}")

------------------------------------------------------------------------------------------------------------------
Calling stakeICX, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x2b5e3af16b1880000', '0x4e6f6e65']} 

{'scoreAddress': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x2b5e3af16b1880000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx2d9ec07136fe5d1a7173e55b0773b41d878db911', 'indexed': ['TokenTransfer(Ad

In [110]:
sICX_amount = 100 * ICX
bnUSD_amount = 200 * ICX
icx_price = 2
sICX_contribute_amount = sICX_amount
bnUSD_contribute_amount = bnUSD_amount

txns = [{'contract': 'staking', 'value': sICX_amount, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': int(icx_price * 4 * bnUSD_amount), 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params({'_owner': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
print(result)
main_debt = int(result["total_debt"],0)/ICX
main_collateral = int(result["collateral"], 0)/ICX

print(f"Debt: {main_debt}, collateral: {main_collateral}")

------------------------------------------------------------------------------------------------------------------
Calling stakeICX, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['IScoreClaimedV2(Address,int,int)', 'cx2d9ec07136fe5d1a7173e55b0773b41d878db911'], 'data': ['0x1fc2c7ae12ca160', '0x8217acc4e89c']} 

{'scoreAddress': 'cx2d9ec07136fe5d1a7173e55b0773b41d878db911', 'indexed': ['IscoreClaimed(int,int)', '0x460c5c', '0x8217acc4e89c'], 'data': []} 

{'scoreAddress': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000', '0x4e6f6e6

In [111]:
send_tx('oracle', 0, 'rig_the_market', {'_base': 'USD', '_quote': 'ICX', '_rate': ICX}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling rig_the_market, with parameters {'_base': 'USD', '_quote': 'ICX', '_rate': 1000000000000000000} on the oracle contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************


{'txHash': '0x70efdba0cecd3bc26372dce9a55e8e1a063ac0e118ecef734c3c21708eb1016b',
 'blockHeight': 4590694,
 'blockHash': '0x88571e8a7d1a10417cc551b355f1259d85beba07bbb3b2dff6d3930b8539d928',
 'txIndex': 1,
 'to': 'cxebb9a5e5abfacccd978c19e38e5dae6da7c6f5cb',
 'stepUsed': 145760,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 145760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [112]:
send_tx('rebalancing', 0, 'rebalance', {}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling rebalance, with parameters {} on the rebalancing contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx0b13453906786489c7989d464f33ed6dc33256bc', 'cx337517d2459fd0abf52e4662bcfb0d872214171b', '0x15065c7922b78000'], 'data': ['0x7b226d6574686f64223a225f73776170222c22706172616d73223a7b22746f546f6b656e223a22637837373362323033323536343931323238616130356166663936666439333537653338386635396566227d7d']} 

{'scoreAddress': 'cx773b203256491228aa05aff96fd9357e388f59ef', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx337517d2459fd0abf52e4662bcfb0d8722

{'txHash': '0x67ea92b82ab0f73018c1e503c7b62f529f00dc74cd536af160fe4870cddda4d4',
 'blockHeight': 4590697,
 'blockHash': '0x8c73dd89694f966e8336502f0cb433093017cb7aac221e6334340fbd48013171',
 'txIndex': 1,
 'to': 'cxf7403ab96d90675a13b2d28c20f021dcc8be6893',
 'stepUsed': 1060320,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1060320,
 'eventLogs': [{'scoreAddress': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx0b13453906786489c7989d464f33ed6dc33256bc',
    'cx337517d2459fd0abf52e4662bcfb0d872214171b',
    '0x15065c7922b78000'],
   'data': ['0x7b226d6574686f64223a225f73776170222c22706172616d73223a7b22746f546f6b656e223a22637837373362323033323536343931323238616130356166663936666439333537653338386635396566227d7d']},
  {'scoreAddress': 'cx773b203256491228aa05aff96fd9357e388f59ef',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx337517d2459fd0abf52e4662bcfb0d872214171b',
    'cx0b13453906786489c7989d464f33ed6dc33256

In [113]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params({'_owner': btest_wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
print(result)
btest_debt_new = int(result["total_debt"],0)/ICX
btest_collateral_new = int(result["collateral"], 0)/ICX


result = call_tx('dex', 'getPoolStats', {'_id': 2})
total_sicx_new = int(result['quote'], 0) / ICX
total_lp_new = int(result['total_supply'], 0)

user_lp_new = int(call_tx('dex', 'balanceOf', {'_owner': btest_wallet.get_address(), '_id': 2}), 0)

btest_sicx_lp_new = total_sicx_new * user_lp_new / total_lp_new

btest_total_sicx_new = btest_collateral_new + btest_sicx_lp_new

print(f"Debt: {btest_debt_new}, collateral: {btest_collateral_new}, lp sicx: {btest_sicx_lp_new}, total: {btest_total_sicx_new}")


{'pos_id': '0x1', 'created': '0x5cea18af53cf0', 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x4106564b4d992d8000', 'bnUSD': '0x56bd2453ba11a5556'}, 'total_debt': '0x33ddb116141a45c6f', 'collateral': '0x410656bee468288950', 'ratio': '0x11661021d692ea32b', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getPoolStats, with parameters {'_id': 2} on the dex contract.
------------------------------------------------------------------------------------------------------------------
{'base': '0x836a956e9940ef800', 'quote': '0x1019d4ab97ed230000', 'base_token': 'cx13d86dfb8b5e6db344a82f823a1259de8d6387a9', 'quote_token': 'cx773b203256491228aa05aff96fd9357e388f59ef', 'total_supply': '0xb7fec6152b435a428', 'price': '0x1b345b4cf7397e19', 'name': None, 'base_decimals': '0x12', 'quote_decimal

In [95]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

{'base': '0x836a956e9940ef800', 'quote': '0x1019d4ab97ed230000', 'base_token': 'cx74eb2b485b811a8d551e68b8cbc0ff9af9f695b4', 'quote_token': 'cxcffd0c80f999a314a1321f56920129f605263b18', 'total_supply': '0xb7fec6152b435a428', 'price': '0x1b345b4cf7397e19', 'name': None, 'base_decimals': '0x12', 'quote_decimals': '0x12', 'min_quote': '0x8ac7230489e80000'}


In [ ]:
# Add assets to existing pool

sICX_contribute_amount = int(2 * ICX)
bnUSD_contribute_amount = int(2 * ICX)

txns = [{'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['baln']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['baln']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 38a

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(5, 10):
    data = "{\"method\": \"_deposit\"}".encode("utf-8")
    params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
    send_tx('baln', 0, 'transfer', params, wallets[i])
    data = "{\"method\": \"_deposit\"}".encode("utf-8")
    params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
    send_tx('bnUSD', 0, 'transfer', params, wallets[i])
    params = {'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}
    send_tx('dex', 0, 'add', params, wallets[i])

In [ ]:
# Cell 38b

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 2 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexPermit")\
    .params({'_id': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**10

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['bnUSD']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['bnUSD']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':2, '_name': 'sICX/bnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':3, '_name': 'BALN/bnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
call_tx('rewards', 'getRecipientsSplit', {})

In [ ]:
# Set the percentages to use with the BALN/bnUSD pool included.

# {
#     "Worker Tokens": "200 000 000 000 000 000",
#     "Reserve Fund": "50 000 000 000 000 000",
#     "DAOfund": "225 000 000 000 000 000",
#     "Loans": "250 000 000 000 000 000",
#     "sICX/ICX": "100 000 000 000 000 000",
#     "sICX/bnUSD": "175 000 000 000 000 000",
#     "BALN/sICX": "0",
#     "BALN/bnUSD": "0",
#     "OMM/IUSDC": "0",
#     "OMM/USDb": "0",
#     "OMM/sICX": "0"
# }

RECIPIENTS = [{'recipient_name': 'Loans', 'dist_percent': 20 * 10**16},
              {'recipient_name': 'sICX/ICX', 'dist_percent': 10 * 10**16},
              {'recipient_name': 'sICX/bnUSD', 'dist_percent': 175 * 10**15},
              {'recipient_name': 'BALN/bnUSD', 'dist_percent': 175 * 10**15},
              {'recipient_name': 'BALN/sICX', 'dist_percent': 5 * 10**16},
              {'recipient_name': 'OMM/IUSDC', 'dist_percent': 0},
              {'recipient_name': 'OMM/USDb', 'dist_percent': 0},
              {'recipient_name': 'OMM/sICX', 'dist_percent': 0},
              {'recipient_name': 'Worker Tokens', 'dist_percent': 20 * 10**16},
              {'recipient_name': 'Reserve Fund', 'dist_percent': 5 * 10**16},
              {'recipient_name': 'DAOfund', 'dist_percent': 5 * 10**16}]

send_tx('governance', 0, 'updateBalTokenDistPercentage', {'_recipient_list': RECIPIENTS}, btest_wallet)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":190}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 45

path = [contracts['bnUSD']['SCORE'], contracts['sicx']['SCORE'], contracts['bnUSD']['SCORE'], contracts['sicx']['SCORE'], None]

path_json = json.dumps(path)


to_token = contracts['bnUSD']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"path\":" + path_json + "}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['router']['SCORE'], '_value': ICX // 100, '_data': data}

print(params)
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [22]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_id':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx951242b96e6bb6b2ef5148ba062dec44b75d1136', 'indexed': ['Remove(int,Address,int,int,int)', '0x2', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0xde0b6b3a7640000'], 'data': ['0xabb3a65d28175e8', '0x11f25dfa8326665b']} 

{'scoreAddress': 'cx951242b96e6bb6b2ef5148ba062dec44b75d1136', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cxf000000000000000000000000000000000000000'], 'data': ['0x2', '0xde0b6b3a7640000']} 

{'scoreAddress': 'cxa2f980a6a8fe637ab0704a92531cb4cf08519170', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx951242b96e6bb6b2ef5148ba062dec44b75d1136', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0xabb3a65d28175e8'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx951242b96e6bb6b2ef5148ba062dec44b75d1136', 'indexed': ['Withdraw(Address,Address,int)', 'cxa2f980a6a8fe637ab0704a92531cb4cf08519170', 'hx3f01840a5

82.32503161168343

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [15]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 0
Failure: {'code': 32, 'message': 'Balanced DEX: Assets must remain in the pool for 24 hours, please try again later.'}


In [ ]:
call_tx('dex', 'getPoolStats', {'_id': 1})

In [ ]:
# address_list = [
#     wallets[1].get_address(),
#     wallets[2].get_address(),
#     wallets[3].get_address(),
#     wallets[4].get_address(),
#     wallets[5].get_address(),
#     wallets[6].get_address(),
#     wallets[7].get_address(),
#     wallets[8].get_address(),
# ]

def query_total_supply(address_list, pool_id, day, minimum):
    snapshot = call_tx('dex', 'totalSupplyAt', {'_account': address, '_id': pool_id, '_snapshot_id': day, '_twa': 1}, 0)
    return int(snapshot, 0)

def get_min_supply(pool_id):
    pool_stats = call_tx('dex', 'getPoolStats', {'_id': pool_id})
    quote_token = pool_stats['quote_token']
    total_quote = int(pool_stats['quote'], 0)
    total_supply = int(pool_stats['total_supply'], 0)

    min_quote = 50 * ICX
    if quote_token == contracts['sicx']['SCORE']:
        min_quote = 25 * ICX
    elif quote_token == contracts['bnUSD']['SCORE']:
        min_quote = 25 * ICX
    
    min_liquidity = min_quote * total_supply // total_quote
    
    return min_liquidity
    

def query_addresses(address_list, pool_id, day):
    minimum = get_min_supply(pool_id)
    rv = {}
    for address in address_list:
        snapshot = call_tx('dex', 'balanceOfAt', {'_account': address, '_id': pool_id, '_snapshot_id': day, '_twa': 0}, 0)
        if int(snapshot, 0) > minimum:
            twa = call_tx('dex', 'balanceOfAt', {'_account': address, '_id': pool_id, '_snapshot_id': day, '_twa': 1}, 0)
            rv[address] = int(twa, 0)
    return rv


In [ ]:
with open("../bad_rewards_fix/correct_rewards.pkl", 'rb') as f:
    pool_awards = pkl.load(f)

In [ ]:
pools = ['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD']
list(pool_awards[pools[2]][1].keys())

In [ ]:
to_token = contracts['baln']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
results = {}
for pool in [1, 2, 3]:
    results[pool] = {}
    for day in [1, 2, 3]:
        address_list = list(pool_awards[pools[pool-1]][day].keys())
        results[pool][day] = query_addresses(address_list, pool, day)

In [ ]:
# with open("../bad_rewards_fix/bad_data_batches.pkl", 'wb') as f:
#     pkl.dump(results, f)

In [ ]:
# We need to filter out the addresses that did not qualify for rewards due to insufficient liquidity provided.
# The results list will be the determinant of whether an address qualifies or not.

total_awards = [3 * [10**22], 3 * [175 * 10**20], 3 * [175 * 10**20]]
mining_total = {}
pool_awards_filtered = {}
for pool in [1, 2, 3]:
    pool_awards_filtered[pool] = {}
    mining_total[pool] = {}
    for day in [1, 2, 3]:
        mining_total[pool][day] = 0
        pool_awards_filtered[pool][day] = {}
        for address in pool_awards[pools[pool-1]][day]:
            if address in results[pool][day]:
                mining_total[pool][day] += pool_awards[pools[pool-1]][day][address]
                pool_awards_filtered[pool][day][address] = pool_awards[pools[pool-1]][day][address]
        for address in pool_awards_filtered[pool][day]:
            share = total_awards[pool-1][day-1] * pool_awards_filtered[pool][day][address] // mining_total[pool][day]
            mining_total[pool][day] -= pool_awards_filtered[pool][day][address]
            total_awards[pool-1][day-1] -= share
            pool_awards_filtered[pool][day][address] = share


In [ ]:
# with open("../bad_rewards_fix/pool_awards_filtered.pkl", 'wb') as f:
#     pkl.dump(pool_awards_filtered, f)

In [ ]:
with open("../bad_rewards_fix/pool_awards_filtered.pkl", 'rb') as f:
    pool_awards_filtered = pkl.load(f)

In [ ]:
bad_totals = {}
for pool in [1, 2, 3]:
    bad_totals[pool] = {}
    for day in [1, 2, 3]:
        bad_totals[pool][day] = sum(results[pool][day].values())

pool_alloc = [10**22, 175 * 10**20, 175 * 10**20]

bad_rewards = {}
for pool in [1, 2, 3]:
    bad_rewards[pool] = {}
    for day in [1, 2, 3]:
        bad_rewards[pool][day] = {}
        for address in results[pool][day]:
            share = pool_alloc[pool-1] * results[pool][day][address] // bad_totals[pool][day]
            bad_rewards[pool][day][address] = share
            pool_alloc[pool-1] -= share
            bad_totals[pool][day] -= results[pool][day][address]

In [ ]:
# with open("../bad_rewards_fix/bad_rewards.pkl", 'wb') as f:
#     pkl.dump(bad_rewards, f)

In [ ]:
with open("../bad_rewards_fix/bad_rewards.pkl", 'rb') as f:
    bad_rewards = pkl.load(f)

In [ ]:
rewards_diff = {}
for pool in [1, 2, 3]:
    rewards_diff[pool] = {}
    for day in [1, 2, 3]:
        rewards_diff[pool][day] = {}
        for address in results[pool][day]:
            rewards_diff[pool][day][address] = pool_awards_filtered[pool][day][address] - bad_rewards[pool][day][address]

In [ ]:
rewards_diff

In [ ]:
# with open("../bad_rewards_fix/rewards_diff.pkl", 'wb') as f:
#     pkl.dump(rewards_diff, f)

In [ ]:
with open("../bad_rewards_fix/rewards_diff.pkl", 'rb') as f:
    rewards_diff = pkl.load(f)

In [ ]:
diff_totals = {}
for pool in [1, 2, 3]:
    diff_totals[pool] = {}
    for day in [1, 2, 3]:
        diff_totals[pool][day] = {'sum': 0, 'neg': 0, 'pos': 0}
        diff_totals[pool][day]['sum'] = sum(rewards_diff[pool][day].values()) / 10**18
        for address in rewards_diff[pool][day]:
            if rewards_diff[pool][day][address] >= 0:
                diff_totals[pool][day]['pos'] += rewards_diff[pool][day][address]
            else:
                diff_totals[pool][day]['neg'] += rewards_diff[pool][day][address]
        diff_totals[pool][day]['pos'] = diff_totals[pool][day]['pos'] / 10**18
        diff_totals[pool][day]['neg'] = diff_totals[pool][day]['neg'] / 10**18

In [ ]:
diff_totals

In [ ]:
totals_owed = {}
for pool in [1, 2, 3]:
    for day in [1, 2, 3]:
        for address in rewards_diff[pool][day]:
            if rewards_diff[pool][day][address] > 0:
                if address not in totals_owed:
                    totals_owed[address] = rewards_diff[pool][day][address]
                else:
                    totals_owed[address] += rewards_diff[pool][day][address]

In [ ]:
with open('bal_alloc2.csv', 'w') as f:
    f.write(f'Address, orig.d1p1, delta.d1p1, total.d1p1, orig.d1p2, delta.d1p2, total.d1p2, d2p1, d2p2, orig.d2p3, delta.d2p3, total.d2p3, d3p1, d3p2, d3p3, total\n')
    for address in totals_owed:
        oamount1 = bad_rewards[1][1].get(address, 0) / 10**18
        oamount2 = bad_rewards[2][1].get(address, 0) / 10**18
        oamount3 = bad_rewards[3][2].get(address, 0) / 10**18
        damount1 = rewards_diff[1][1].get(address, 0) / 10**18
        damount2 = rewards_diff[2][1].get(address, 0) / 10**18
        damount3 = rewards_diff[3][2].get(address, 0) / 10**18
        tamount1 = damount1 + oamount1 if damount1 > 0 else oamount1
        tamount2 = damount2 + oamount2 if damount2 > 0 else oamount2
        tamount3 = damount3 + oamount3 if damount3 > 0 else oamount3
        f.write(f'{address}, '
                f'{oamount1}, '
                f'{damount1}, '
                f'{tamount1}, '
                f'{oamount2}, '
                f'{damount2}, '
                f'{tamount2}, '
                f'{rewards_diff[1][2].get(address, 0) / 10**18}, '
                f'{rewards_diff[2][2].get(address, 0) / 10**18}, '
                f'{oamount3}, '
                f'{damount3}, '
                f'{tamount3}, '
                f'{rewards_diff[1][3].get(address, 0) / 10**18}, '
                f'{rewards_diff[2][3].get(address, 0) / 10**18}, '
                f'{rewards_diff[3][3].get(address, 0) / 10**18}, '
                f'{totals_owed[address] / 10**18}\n')

### len(totals_owed) = 1607

### sum(totals_owed.values()) / 10**18 = 81252.796380166532499864

In [ ]:
batches = []
addresses = []
amounts = []
for address in totals_owed:
    addresses.append(address)
    amounts.append(totals_owed[address])
    if len(addresses) % 250 == 0:
        batches.append((addresses[:], amounts[:]))
        addresses = []
        amounts = []
batches.append((addresses[:], amounts[:]))

In [ ]:
total = 0
for batch in batches:
    print(f'Addresses length: {len(batch[0])}, amounts length: {len(batch[1])}, amounts sum: {sum(batch[1])}')
    total += sum(batch[1])
print(f'Total distribution = {total / 10**18} BALN.')

In [ ]:
# with open("../bad_rewards_fix/totals_owed.pkl", 'wb') as f:
#     pkl.dump(totals_owed, f)

In [ ]:
# with open("../bad_rewards_fix/batches.pkl", 'wb') as f:
#     pkl.dump(batches, f)

### The above sums to a discrepancy of 72500 undistributed BALN plus ~8752.796 incorrectly allocated.
Founding team will supply 8754 BALN to be distributed on the rewards SCORE. All Balanced Participants will then be paid everything they are owed plus some bonus for early paritipant appreciation.

In [ ]:
# Set of DEX addresses
address_set = set()
for pool in [1, 2, 3]:
    for day in [1, 2, 3]:
        address_set.update(set(pool_awards[pools[pool-1]][day].keys()))

In [ ]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

borrowerCount = int(call_tx('loans', 'borrowerCount', {}, False), 0)

positions = {}
addresses = address_set
for i in range(1, borrowerCount + 1):
    position = call_tx('loans', 'getPositionByIndex', {'_index': i, '_day': -1}, False)
    addresses.add(position['address'])
    positions[position['address']] = position

holders = call_tx('rewards', 'getBalnHoldings', {'_holders': list(addresses)}, False)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    result = int(call_tx('baln', 'balanceOf', {'_owner': contracts[contract]['SCORE']}, False), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

holdings = {key: {'rewards': [int(holders[key], 0),
                              int(holders[key], 0) / 10**18]} for key in holders}

for address in holdings:
    holdings[address]['wallet'] = [int(call_tx('baln', 'balanceOf', {'_owner': address}, False), 0)]
    holdings[address]['wallet'].append(holdings[address]['wallet'][0] / 10**18)

total_rewards = 0
total_wallet = 0
for key in holdings:
    total_rewards += holdings[key]['rewards'][0]
    total_wallet += holdings[key]['wallet'][0]
    print(f"{key}: {holdings[key]['wallet']}, {holdings[key]['rewards']}")

total_supply = int(call_tx('baln', 'totalSupply', {}, False), 0)
total_moved = total_supply - total_wallet - sum(baln_balances.values())

print(f'Total unclaimed: {total_rewards / 10**18}')
print(f'Total in borrower wallets: {total_wallet / 10**18}')
print(f'Total moved to non participant wallets: {total_moved / 10**18}')
print(baln_balances)
print(f'Total on contracts: {total_balances / 10**18}')
print(total_supply)

## Allocate BALN on rewards SCORE

In [ ]:
# send ICX to cover costs

send_icx(bdist_wallet.get_address(), 4 * ICX, btest_wallet)

In [ ]:
# Transfer 8754 BALN to the rewards SCORE.

send_tx('baln', 0, 'transfer', {'_to': 'cx10d59e8103ab44635190bd4139dbfd682fa2d07e', '_value': 8754 * ICX}, bdist_wallet)

In [ ]:
# Allocate reward BALN to addresses in batches.

for batch in batches:
    send_tx('governance', 0, 'bonusDist', {'_addresses': batch[0], '_amounts': batch[1]}, btest_wallet)
    sleep(4)

In [ ]:
batches

In [ ]:
baln_holdings = {}
def bonusDist(addresses, amounts):
    for i, address in enumerate(addresses):
        if address not in baln_holdings:
            baln_holdings[str(address)] = amounts[i]
        else:
            print(f'Should not be here.')
            baln_holdings[str(address)] += amounts[i]

In [ ]:
for batch in batches:
    bonusDist(batch[0], batch[1])

In [ ]:
sum(baln_holdings.values()) / 10**18

In [ ]:
for i in range(10):
    fast_send_tx('rewards', 0, 'distribute', {}, btest_wallet)